<a href="https://colab.research.google.com/github/dashatenoff/recsys-vk/blob/main/notebooks/content_unsupervised_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unsupervised content-based recommendation (user profile)

В данном эксперименте реализована **unsupervised контентная рекомендательная модель**
на основе эмбеддингов видеоклипов.

Для каждого пользователя строится **профиль пользователя** как среднее (mean pooling)
эмбеддингов всех клипов, с которыми пользователь положительно взаимодействовал
(`timespent > 5`) в обучающей выборке.

Рекомендации формируются следующим образом:
- для пользователя строится вектор-профиль;
- вычисляется cosine similarity между профилем пользователя и эмбеддингами всех клипов;
- пользователю рекомендуются топ-N клипов с наибольшей косинусной близостью.

Модель является полностью **unsupervised**:
- не используется информация о будущих взаимодействиях,
- не применяется обучение модели или оптимизация под целевую метрику,
- используется только контентная информация (эмбеддинги клипов).

Цель эксперимента — проверить,
насколько хорошо эмбеддинги отражают семантическую близость контента
и могут ли они использоваться для рекомендаций без обучения.


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from google.colab import drive
import os

In [3]:
drive.mount('/content/drive')
os.listdir('/content/drive/MyDrive')
train = pd.read_parquet('/content/drive/MyDrive/VK/train.parquet')
test = pd.read_parquet('/content/drive/MyDrive/VK/test.parquet')
item_embeddings = pd.read_parquet('/content/drive/MyDrive/VK/item_embeddings.parquet')
item_metadata = pd.read_parquet('/content/drive/MyDrive/VK/item_metadata.parquet')
user_metadata = pd.read_parquet('/content/drive/MyDrive/VK/user_metadata.parquet')

Mounted at /content/drive


In [4]:
item_metadata.head()

,item_id,author_id,duration,train_interactions_rank,embedding
0,326091735,116090,89,17367,"[-0.5078125, 0.044647216796875, 0.447021484375..."
1,337826988,120666,37,8281,"[-0.253173828125, 0.128173828125, 0.2349853515..."
2,582660968,125834,167,12133,"[-0.59814453125, -0.1922607421875, 0.017166137..."
3,223344189,127291,90,17943,"[-0.343994140625, 0.053680419921875, 0.1712646..."
4,413392655,127581,99,18253,"[-0.54541015625, 0.035614013671875, 0.05685424..."


In [5]:
user_metadata.head()

,user_id,age,gender,geo,train_interactions_rank
0,202612548,18,1,1,965
1,189035614,18,1,13,6457
2,22320303,18,1,14,8919
3,194699221,18,1,16,2424
4,392744532,18,1,17,3383


In [6]:
train.head()
test.head()
item_embeddings.head()

,item_id,embedding
0,187690,"[-0.61474609375, -0.265869140625, 0.3693847656..."
1,566082,"[-0.28955078125, 0.170166015625, -0.0369262695..."
2,1013885,"[-0.51904296875, 0.297119140625, 0.12939453125..."
3,1144655,"[-0.34375, -0.07366943359375, 0.21337890625, 0..."
4,1222720,"[-0.55859375, -0.321533203125, 0.2030029296875..."


In [7]:
train['rating'] = train['timespent'] > 5
test['rating'] = test['timespent'] > 5
train_pos = train[train['rating']]


item_to_embedding = {
    row.item_id : np.array(row.embedding)
    for row in item_embeddings.itertuples()
}

some_item = list(item_to_embedding.keys())[0]
item_to_embedding[some_item].shape


(32,)

#Строим историю пользователя

In [8]:
user_history = train_pos.groupby('user_id')['item_id'].apply(list)
user_history.head()

,item_id
user_id,
20125,"[337132618, 445428097, 579126608, 188415006, 5..."
32170,"[253099749, 60779675, 377529740, 280199908, 77..."
33549,"[327484273, 588664675, 483409790, 578259279, 1..."
48637,"[21625794, 402842289, 424493738, 190285675, 19..."
124907,"[191602344, 545325922, 470819572, 337132618, 8..."


In [9]:
def build_user_embedding(items_id):
  vectors = [
      item_to_embedding[i]
      for i in items_id
      if i in item_to_embedding]

  if len(vectors) == 0:
    return None
  return np.mean(vectors, axis=0)

build_user_embedding(user_history.iloc[0])


array([-4.65937720e-01, -7.50238366e-02,  1.23224699e-01, -2.23737790e-02,
       -1.10847688e-01, -4.16370026e-02, -3.44760579e-02,  3.52473342e-02,
       -2.04110344e-02, -1.49675550e-02, -3.16310161e-02, -1.59212831e-02,
       -2.83909231e-02,  1.10615796e-02,  8.65245486e-03,  2.06013769e-02,
        1.07035047e-02, -1.77472412e-02,  1.63744407e-02, -3.16919055e-03,
       -4.10758985e-02,  9.05988572e-03,  1.98076434e-02, -1.14918484e-02,
        2.20331930e-02, -5.97191912e-03, -1.26597323e-02,  9.97077922e-03,
        8.03470612e-05, -4.65841922e-03,  7.99861219e-04, -1.21736113e-02])

In [10]:
all_item_ids = item_embeddings['item_id'].values

all_item_embeddings = np.vstack( item_embeddings['embedding'].apply(np.array).values )

all_item_embeddings.shape

(4398, 32)

#Считаем cosine similarity

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_content(user_id, k=10):
  if user_id not in user_history:
    return []

  items_id = user_history[user_id]
  user_emb = build_user_embedding(items_id)
  sims = cosine_similarity(
    user_emb.reshape(1, -1),
    all_item_embeddings
  )
  sims = sims[0]
  top_ind = np.argsort(-sims)[:k]
  return all_item_ids[top_ind]

recommend_content(train['user_id'].iloc[0])


array([550674033, 531063416, 320079060,  45011104, 439563707, 544642404,
       458420819, 333325493,  96538469, 256873871], dtype=uint32)

#Получаем рекомендации для всех пользователей


In [12]:
recs = []
test_user = test['user_id'].unique()

for user_id in test_user:
  recs_items = list(recommend_content(user_id))
  for item_id in recs_items:
    recs.append({
        'user_id' : user_id,
        'recs' : item_id
    })
submission = pd.DataFrame(recs)
submission.to_csv('submission_unsupervised' , index=False)


In [13]:
submission.head(20)

,user_id,recs
0,506947605,550674033
1,506947605,439563707
2,506947605,45011104
3,506947605,96538469
4,506947605,284345673
5,506947605,176196469
6,506947605,531063416
7,506947605,7319207
8,506947605,256873871
9,506947605,130976514


#MAP@10

In [16]:
aps = []
rel = (
    test[test['rating']]
    .groupby('user_id')['item_id']
    .apply(set)
)

pred = submission.groupby('user_id')['recs'].apply(list)


for user_id in pred.index:
  if len(rel.get(user_id, set())) == 0:
    continue
  ord = 1
  cor = 0
  score = 0.0
  for item_id in pred[user_id]:
    if item_id in rel.get(user_id, set()):
      cor+=1
      score+=cor/ord
    ord+=1
  aps.append(score/min(len(rel.get(user_id, [])), 10))
map10 = sum(aps)/len(aps)
map10

0.0033364771893520484

### Различие между in-sample и out-of-sample оценкой

Для данной модели наблюдается существенный разрыв между
качеством на train и test выборках:

- Train MAP@10 ≈ 0.043
- Test  MAP@10 ≈ 0.0033

Это подтверждает, что:
- эмбеддинги содержат корректную семантическую информацию,
- cosine similarity адекватно отражает близость контента,
- однако unsupervised модель не обладает предсказательной способностью
  в отношении будущих взаимодействий.

Подобное поведение является типичным для content-based моделей
без обучения и дополнительного контекстного или коллаборативного сигнала.
### Ограничения unsupervised content-based подхода

Реализованная модель использует только контентные эмбеддинги
и не оптимизируется под предсказание будущего взаимодействия.
Это приводит к следующим ограничениям:

1. Модель не учитывает популярность контента  
   — редкие, но семантически похожие видео
     рекомендуются так же часто, как и популярные.

2. Модель не учитывает временную динамику  
   — интересы пользователя могут меняться,
     а усреднённый user profile плохо отражает краткосрочные предпочтения.

3. Оптимизация происходит под семантическую близость,
   а не под вероятность конкретного взаимодействия,
   что особенно критично для метрики MAP@10.

В результате модель хорошо восстанавливает
ранее просмотренный пользователем контент (in-sample),
но слабо предсказывает будущие взаимодействия (out-of-sample).
